# Trabalho Prático 3

## Introdução

A resolução deste trabalho prático tem 3 objetivos principais:

- Criar uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ .
- Implementação de um esquema de assinaturas digitais _FALCON_ .
- Estudar gamas de valores de determinados parâmetros que tornem viável um ataque de inversão de chave pública ou inversão do criptograma utilizando redução de bases.

Este relatório está, desta forma, dividido em três partes, cada parte correspondente à resolução de um dos problemas e, além disso, está estruturado de forma a que o texto entre os _snippets_ de código seja suficientemente explicativo sobre a implementação e desenho da solução em cada um dos problemas.

## Algoritmo de Boneh & Venkatesan

Esta secção tem como propósito definir uma classe **Python** que implemente o algoritmo de _Boneh & Venkatesan_ , que explora o _Hidden Number Problem_ que, se bem sucedido, permite obter um segredo a partir de um conjunto de dados. Além disso, a secção está dividida em duas partes:

- A primeira parte define apenas o algoritmo de _Boneh & Venkatesan_ .
- A segunda parte, apelidade de teste, fornece os parâmetros necessários à instância do algoritmo para que ele descubra o segredo **s** a partir dos mesmos.

**CONVÉM DEFINIR UM BOCADO O HNP AQUI. DIZER O QUE É.**

### Definição do oráculo HNP

O propósito desta secção passa por definir o oráculo que gera um segredo e, posteriormente, retorna o vetor u, com l elementos, onde cada um é um inteiro representativo dos k bits mais significativos de $ s * xi $.

In [11]:
class HNPOracle:
    
    def __init__(self,p):
        # gerar o segredo.
        self.secret = ZZ.random_element(p)
        print 'secret'
        print self.secret
    
    def calculate_u_vector(self,x_vector,k,p):
        # k - nr de bits significativos do produto entre x e self.secret mod p
        # calcular o vetor u.
        u_vector = []
        for xi in x_vector:
            sxi = ZZ(Mod(self.secret*xi,p))
            sxi = sxi.digits(2)
            sxi.reverse()
            sxi_string = ''
            if len(sxi) < k:
                for i in range(0,len(sxi)):
                    sxi_string = sxi_string + str(sxi[i])
            else:
                for i in range(0,k):
                    sxi_string = sxi_string + str(sxi[i])
            u_vector.append(ZZ(int(sxi_string,2)))
        return u_vector
            
        
    def compare_secret(self,calculated_secret):
        # comparar o segredo calculado com o segredo gerado
        if self.secret == calculated_secret:
            return True
        else:
            return False

### Definição da função de geração do vetor aleatório

In [12]:
def generate_l_random_elements(l,p):
    x = []
    for i in range(0,l):
        x.append(ZZ.random_element(p))
    return x

### Definição do algoritmo de Boneh & Venkatesan

In [39]:
import sage.modules.free_module_integer as fmi
import numpy as np

class BV:
    
    def __init__(self,u,x,k,p,l):
        # construir a matriz L , lambda e o target T
        self.param_lambda = 2^(k+1)
        self.L = self.param_lambda * p * matrix.identity(l)
        self.L = self.L.transpose()
        self.L = self.L.insert_row(l,zero_vector(l))
        self.L = self.L.transpose()
        temp_x = [self.param_lambda * i for i in x]
        temp_x.append(1)
        self.L = self.L.insert_row(l,temp_x)
        self.target = [self.param_lambda * i for i in u]
        self.target.append(0)
        
    
    def solve(self,x,p,l):
        L = fmi.IntegerLattice(self.L).reduced_basis
        L = matrix(L)
        print 'reduced L lattice...'
        self.target = matrix(self.target)
        t = matrix(1,l+1,list(-self.target))
        zero = matrix(l+1,1,[0]*(l+1))
        M = matrix(1,1,ZZ.random_element(p))
        L1 = block_matrix(2,2,[[L,zero],[t,M]])
        print 'created the matrixes...'
        ret = fmi.IntegerLattice(L1).reduced_basis
        error1 = np.array(ret[l+1][:-1])
        y1 = error1 + self.target
        print ' calculated parameters...'
        print 'error'
        print error1
        print 'target'
        print self.target
        print 'vector'
        print y1
        print y1[0][l]
        

### Teste ao algoritmo de Boneh & Venkatesan

In [46]:
# Estes parâmetros têm que ser substituídos pelos verdadeiros.
# x possui l elementos gerados aleatoriamente entre 0 e p-1.
# u possui l elementos, onde cada ui é igual ao inteiro representativo dos k bits mais significativos de s*xi
# Ver valores de l,k e p que sejam realistas e bons para resolver.
# Calcular os vetores x (aleatorio) e u(calculado a partir do aleatorio com o segredo.)
l = 2^7
k = 24
p = 2^32
x_vector = generate_l_random_elements(l,p)
oracle = HNPOracle(p)
print 'finished initializing oracle...'
u_vector = oracle.calculate_u_vector(x_vector,k,p)
print 'calculated u_vector..'
bv = BV(u_vector,x_vector,k,p,l)
print 'finished initializing boneh & venkatesan algorithm...'
bv.solve(x_vector,p,l)
print 'solved hnp...'

secret
4056811883
finished initializing oracle...
calculated u_vector..
finished initializing boneh & venkatesan algorithm...
reduced L lattice...
created the matrixes...
 calculated parameters...
error
[  7540200964096000   2499932556623872 -34591306562928640
  43526291311296512  68312551133282304 -53282993497702400
  -6738787715514368  58265258908712960  49339091392659456
 -58453540007313408 -70362040673239040  59917767104528384
  29560766659035136 -26118577333469184  -8632618312531968
  42667858815090688  34917335517102080 -64212921432932352
 -62072339461832704   8569239023648768 -38523315702202368
  19032278042673152  28269867731779584   2546888964308992
  -5403151603597312  24380243477266432  29212166041108480
   7540775885733888  47681923559981056 -64270252602556416
  61461924952408064  47831194678591488 -15891798962470912
   5505813066022912 -32869340088893440 -22618180329406464
 -37601042677891072  29986612330889216 -34872652690620416
 -41643246518009856   2729325484834816 -142

## Definição do Esquema NTRU-Encrypt

In [24]:
name        = "NTRU_PKE_443"
d           = 115
N           = 443
p           = 3
q           = next_prime(p*N)       #2048
Z.<x>       = ZZ[]
Q.<x>       = PolynomialRing(GF(q),name='x').quotient(x^N-1)

def vec():
    return  [choice([-1,0,1]) for k in range(N)]

def qrnd(f):
    qq = (q-1)//2 ; ll = map(lift,f.list())
    return [n if n <= qq else n - q  for n in ll]

def prnd(l):
    pp = (p-1)//2
    rr = lambda x: x if x <= pp else x - p        
    return [rr(n%p) if n>=0 else -rr((-n)%p) for n in l]

class NTRU:
    
    def __init__(self):
        print name
        print d
        print N
        print p; print is_prime(p)
        print q; print is_prime(q)
        print Z
        print Q
    
    def keypair(self):
        f = Q(0)
        while not f.is_unit():
            F = Q(vec()); f = 1 + p*F
        G = Q(vec()) ; g = p*G
        self.f = f
        self.h = f^(-1) * g
    
    def encrypt(self, m):
        r = Q(vec())
        return r*self.h + Q(m)
    
    def decrypt(self,e):
        a = e*self.f
        return prnd(qrnd(a))
    


### Teste ao esquema NTRU

In [25]:
K = NTRU()
K.keypair()
m = vec()
e = K.encrypt(m)
print m == K.decrypt(e)


NTRU_PKE_443
115
443
3
True
1361
True
Univariate Polynomial Ring in x over Integer Ring
Univariate Quotient Polynomial Ring in x over Finite Field of size 1361 with modulus x^443 + 1360
True


## Conclusão

## Referências